Define the uncertainties,levers and outcomes.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
import platypus

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome,
                           perform_experiments, ema_logging,Policy,MultiprocessingEvaluator,SequentialEvaluator)
from ema_workbench.em_framework.evaluators import BaseEvaluator

from ema_workbench.em_framework.evaluators import Samplers

from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.salib_samplers import get_SALib_problem


from problem_formulation import get_model_for_problem_formulation

#from SALib.analyze import sobol

C:\Users\aspas\miniconda3\envs\gds\lib\site-packages\ema_workbench\analysis\prim.py:30: ImportWarning: altair based interactive inspection not available
  warnings.warn(


In [2]:
# setting up the model as usual
ema_logging.log_to_stderr(ema_logging.INFO)

model, planning_steps = get_model_for_problem_formulation(2)

In [ ]:
with MultiprocessingEvaluator(model) as evaluator:
    results1 = evaluator.optimize(nfe=1, searchover='levers',
                                 epsilons=[0.1,]*len(model.outcomes))

In [ ]:
from ema_workbench.analysis import parcoords

data = results1.loc[:, [o.name for o in model.outcomes]]
limits = parcoords.get_limits(data) 
limits.loc[0, ['All Costs', 'Expected Number of Deaths']] = 0   #more outcomes to be completed 

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(data)
paraxes.invert_axis('Expected Number of Deaths')
plt.show()

In [ ]:
with MultiprocessingEvaluator(model) as evaluator:
    results2 = evaluator.optimize(nfe=5e3, searchover='levers',
                                 epsilons=[0.01,]*len(model.outcomes))

In [ ]:
data = results2.loc[:, [o.name for o in model.outcomes]]
limits = parcoords.get_limits(data)
limits.loc[0, ['All Costs','Expected Number of Deaths' ]] = 0   #more outcomes to be completed 

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(data)
paraxes.invert_axis('Expected Number of Deaths')
plt.show()